# Deploying an already trained model from PMML into IBM Watson Machine Learning
## in a notebook in Watson Studio Local

See also: <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener noreferrer">Watson Machine Learning Python client</a>


### Step 1: Authenticate with the Watson Machine Learning Python client

In the following cell, paste three pieces of information:

<ul>
<li><b>url</b> : The base URL of your Watson Studio Local server
<p>For example, if the URL of your Watson Studio Local server is:<br/><pre>https://111.22.333.444:5555/home?context=wdp</pre></p>
<p>Then the "url" in the following cell should be:<br/><pre>https://111.22.333.444:5555</pre></p></li>
</ul>
<ul>
<li><p><b>username</b> : The username that you use to sign in to Watson Studio Local</p></li>
</ul>
<ul>
<li><p><b>password</b> : The password that you use to sign in to Watson Studio Local</p></li>
</ul>

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "wml_local",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "2.5.0"
}

client = WatsonMachineLearningAPIClient(wml_credentials)

### Step 2: Download a sample PMML file to the notebook working directory

**About the sample PMML file and model**

The sample PMML file was generated by this notebook: <a href="https://github.com/pmservice/wml-sample-models/blob/master/spark/import-pmml/spark-mllib-export-pmml-scala.ipynb" target="_blank" rel="noopener noreferrer">Saving a Spark MLlib model in PMML format</a>

The sample model is a logistic regression model for predicting whether or not a customer will purchase a tent from a fictional outdoor equipment store, based on the customer charateristics.

The data used to train the model is the "GoSales.csv" training data in the IBM Watson Studio community: <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/aa07a773f71cf1172a349f33e2028e4e" target="_blank" rel="noopener noreferrer">GoSales sample data</a>

In [ ]:
!pip install wget

In [ ]:
import wget
pmml_file_url = 'https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/import-pmml/spark-mllib-lr-model-pmml.xml'
pmml_filename = wget.download( pmml_file_url )
print( pmml_filename )

In [ ]:
!cat spark-mllib-lr-model-pmml.xml

In [ ]:
client.version

### Step 3: Store the model in the project

In [ ]:
# get the Project ID and set the location to save the model to the project
project_id = os.environ['PROJECT_ID']
client.set.default_project(project_id)

In [ ]:
metadata = {
    client.repository.ModelMetaNames.NAME: "Model from PMML",
    client.repository.ModelMetaNames.TYPE: "pmml_4.2",
    client.repository.ModelMetaNames.RUNTIME_UID: "pmml_4.2.1"
}

model_details = client.repository.store_model( model=pmml_filename, meta_props=metadata )

**At this point you can switch to the Project view and complete online deployment manually.  Alternatively, you can choose to programmatically store the model into the deployment space and deploy the model**

### Step 4: Store the model in the deployment space


Use an existing deployment space that is already associated with this project, or create a new deployment space if there is no associated deployment space

In [ ]:
# get project info
import requests

r = requests.get(os.environ['RUNTIME_ENV_APSX_URL'] + "/v2/projects/" + os.environ['PROJECT_ID'], headers = {"Authorization": "Bearer " + os.environ['USER_ACCESS_TOKEN']})

In [ ]:
# if a deployment is assocaited with this project, get deployment space_id

space_id = None 

try:

    wmlProjectCompute = [c for c in r.json()["entity"]["compute"] if c["type"] == "machine_learning"][0]
    
except: 
    # do nothing if there is no associated deplotment space
    print("There is no deployment space associted with this project")

else:
    space_id = wmlProjectCompute["properties"]["space_guid"]
    

In [ ]:
str(space_id)

#### <font color='red'>Action required:</font> If this project is not already associated with a "_Deployment Space_", specify values for the space_name and space_tag in the code cell below

In [ ]:
if str(space_id)=='None':
    space_name = 'XXXXX '  # e.g deployment-space-sidneyp-sandbox 
    space_tag =  'XXXXX'   # e.g deployment-space-tag-sidneyp-sandbox
    
    # create the space and set it as default
    space_meta_data = {
            client.spaces.ConfigurationMetaNames.NAME : space_name,
            client.spaces.ConfigurationMetaNames.TAGS : [{'value': space_tag}]
    }

    stored_space_details = client.spaces.store(space_meta_data)

    space_uid = stored_space_details['metadata']['guid']

    # set the newly created deployment space as the default
    client.set.default_space(space_uid)
    
    

else:
    # retrieve existing space details
    stored_space_details = client.spaces.get_details(space_id)
    space_uid = stored_space_details['metadata']['guid']
    # set deployment space as the default
    client.set.default_space(space_uid)
    


In [ ]:
metadata = {
    client.repository.ModelMetaNames.NAME: "Model from PMML",
    client.repository.ModelMetaNames.TYPE: "pmml_4.2",
    client.repository.ModelMetaNames.RUNTIME_UID: "pmml_4.2.1"
}

In [ ]:
model_details = client.repository.store_model( model=pmml_filename, meta_props=metadata )

### Step 5: Deploy the model

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Model deployment from PMML - code",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [ ]:
model_id = model_details["metadata"]["guid"]
deployment_details = client.deployments.create( artifact_uid=model_id, meta_props=metadata )

### Step 6: Send sample payload data to make a prediction

In [ ]:
deployment_id = deployment_details["metadata"]["guid"]

In the <a href="https://github.com/pmservice/wml-sample-models/blob/master/spark/import-pmml/spark-mllib-export-pmml-scala.ipynb" target="_blank" rel="noopener noreferrer">sample notebook that builds the model and saves it in PMML format</a>, you can see examples of sample data:

Features of a customer who did not buy a tent:
```
| GENDER | AGE | MARITAL_STATUS | PROFESSION     | feature_vector           |
+--------+-----+----------------+----------------+--------------------------+
| "F"    | 35  | "Married"      | "Professional" | [ 1.0, 35.0 , 0.0, 1.0 ] |
```

Features of a customer who did buy a tent:
```
| GENDER | AGE | MARITAL_STATUS | PROFESSION     | feature_vector           |
+--------+-----+----------------+----------------+--------------------------+
| "M"    | 20  | "Single"       | "Sales"        | [ 0.0, 20.0, 1.0, 2.0 ]  |
```

It is the four numeric values in `feature_vector` column that are sent to the model to make a prediction.

The values in the `feature_vector` column for the "GENDER", "MARITAL_STATUS", and "PROFESSION" columns are generated by string indexers:
```
+------+------------+      +--------------+--------------------+      +------------+----------------+
|GENDER|GENDER_index|      |MARITAL_STATUS|MARITAL_STATUS_index|      |  PROFESSION|PROFESSION_index|
+------+------------+      +--------------+--------------------+      +------------+----------------+
|     M|         0.0|      |       Married|                 0.0|      |       Other|             0.0|
|     F|         1.0|      |        Single|                 1.0|      |Professional|             1.0|
+------+------------+      |   Unspecified|                 2.0|      |       Sales|             2.0|
                           +--------------+--------------------+      |   Executive|             3.0|
                                                                      |      Trades|             4.0|
                                                                      | Hospitality|             5.0|
                                                                      |     Student|             6.0|
                                                                      |      Retail|             7.0|
                                                                      |     Retired|             8.0|
                                                                      +------------+----------------+
```

In [ ]:
negative_example_payload = { "input_data" : [ { "fields" : [ "field_0", "field_1", "field_2", "field_3" ], "values" : [ [ 1.0, 35.0, 0.0, 1.0 ] ] } ] }
positive_example_payload = { "input_data" : [ { "fields" : [ "field_0", "field_1", "field_2", "field_3" ], "values" : [ [ 0.0, 20.0, 1.0, 2.0 ] ] } ] }

In [ ]:
client.deployments.score( deployment_id, negative_example_payload )

In [ ]:
client.deployments.score( deployment_id, positive_example_payload )

## Summary
In this notebook, you imported a model into Watson Machine Learning from a sample PMML file using the Watson Machine Learning Python client.

Authors

- **Sarah Packowski** is a member of the IBM Watson Studio Content Design team in Canada.
- Modified for CP4D by **Elena Lowery**

<hr>

Copyright &copy; IBM Corp. 2019. This notebook and its source code are released under the terms of the MIT License.